# Data Cleaning

Note: There is an old script called `data-cleaning-old.ipynb`. This is old and not used anymore.

In [1]:
import numpy as np


import scipy.stats 
import pandas as pd
import pickle

## Game Data

In [2]:
# This function is to convert the string into int
def Str2float(string):
    try:
        return float(string)
    except:
        return "NaN"

In [3]:
#Construct the game stats with each team rosters
#import 2016
game16_df = pd.read_csv('raw-data/15-16game_stats.csv')[["Visitor/Neutral", "PTS", "Home/Neutral", "PTS.1"]]
game16_df['Year'] = 2016
game16_df['PTS'] = game16_df['PTS'].map(lambda x: Str2float(x))
game16_df['PTS.1'] = game16_df['PTS.1'].map(lambda x: Str2float(x))

# get rid of nan or nonsensical values
game16_df = game16_df[game16_df['PTS'].map(lambda x: type(x) is float)]

# difference between visitor and home
game16_df['diff'] = game16_df['PTS'] - game16_df['PTS.1']
game16_df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Year,diff
1,Detroit Pistons,106,Atlanta Hawks,94,2016,12
2,Cleveland Cavaliers,95,Chicago Bulls,97,2016,-2
3,New Orleans Pelicans,95,Golden State Warriors,111,2016,-16
4,Philadelphia 76ers,95,Boston Celtics,112,2016,-17
5,Chicago Bulls,115,Brooklyn Nets,100,2016,15


In [4]:
game16_df.to_csv("clean-data/game_outcomes_15-16.csv")

## Player Data

In [5]:
#offensive Rating
player_df2 = pd.read_csv('raw-data/15-16Per100Pos_stats.csv')[['Player','ORtg']]
player_df2 = player_df2[player_df2['ORtg'] != "ORtg"]
player_df2 = player_df2[~pd.isnull(player_df2['ORtg'])] 
player_df2['ORtg'] = player_df2['ORtg'].map(lambda x: int(x))
player_df2 = pd.DataFrame(player_df2.groupby('Player')['ORtg'].max())
player_df2["Player"] = player_df2.index
player_df2 = player_df2.reset_index(drop=True)
player_df2.head()

,ORtg,Player
0,99,Aaron Brooks
1,114,Aaron Gordon
2,77,Aaron Harrison
3,81,Adreian Payne
4,113,Al Horford


In [7]:
#defensive Rating 
player_d = pd.read_csv('raw-data/15-16Per100Pos_stats.csv')[['Player','DRtg']]
player_d = player_d[player_d['DRtg'] != "DRtg"]
player_d = player_d[~pd.isnull(player_d['DRtg'])] 
player_d['DRtg'] = player_d['DRtg'].map(lambda x: int(x))
player_d = pd.DataFrame(player_d.groupby('Player')['DRtg'].max())
player_d["Player"] = player_d.index
player_d = player_d.reset_index(drop=True)
player_d.head()

,DRtg,Player
0,109,Aaron Brooks
1,105,Aaron Gordon
2,103,Aaron Harrison
3,108,Adreian Payne
4,101,Al Horford


In [8]:
#import name and their teams 
playerTeam_df = pd.read_csv('raw-data/salary.csv', header = 1)[['Player','Tm','2015-16']]

# get rid of nonsensical values
playerTeam_df = playerTeam_df[playerTeam_df['Tm'] != 'Tm']
playerTeam_df = playerTeam_df[playerTeam_df['Player'] != 'Player']
playerTeam_df = playerTeam_df[playerTeam_df['2015-16'] != 'Salary']
# convert to integer
playerTeam_df['2015-16'] = playerTeam_df['2015-16'].map(lambda x: int(x))
# only select one salary per player
salary_df = pd.DataFrame(playerTeam_df.groupby('Player')['2015-16'].max())
salary_df["Player"] = salary_df.index
salary_df = salary_df.reset_index(drop=True)

playerTeam_df = playerTeam_df[~pd.isnull(playerTeam_df['Player'])]
playerTeam_df =  playerTeam_df.drop_duplicates('Player')

# merge
playerTeam_df = playerTeam_df[["Player","Tm"]].merge(salary_df,how="inner", on="Player")
playerTeam_df.head()

,Player,Tm,2015-16
0,Kobe Bryant,Los Angeles Lakers,25000000
1,Joe Johnson,Brooklyn Nets,24894863
2,LeBron James,Cleveland Cavaliers,22971000
3,Carmelo Anthony,New York Knicks,22875000
4,Dwight Howard,Houston Rockets,22359364


In [9]:
# merge all the player info
player_info_df = playerTeam_df.merge(player_d,how='inner',on='Player').merge(player_df2,how='inner',on='Player')
player_info_df
player_info_df.head()

,Player,Tm,2015-16,DRtg,ORtg
0,Kobe Bryant,Los Angeles Lakers,25000000,113,95
1,Joe Johnson,Brooklyn Nets,24894863,114,114
2,LeBron James,Cleveland Cavaliers,22971000,103,116
3,Carmelo Anthony,New York Knicks,22875000,107,107
4,Dwight Howard,Houston Rockets,22359364,104,111


In [10]:
# Save the info to CSV format
player_info_df.to_csv("clean-data/player_info.csv")

The `2015-2016` column refers to the salary.

## Masking

The following part is guest/host_team_line_up. `guest_team_line_up` and `host_team_line_up` are lists of boolean Numpy arrays. Each element of each list corresponds to one matchup in the `game16_df` table. `guest_team_line_up` can be used to select the players from the `player_info_df` table that participated on the guest team, and `host_team_line_up` can be used to select the players from `player_info_df` that participated for the host.

In [11]:
players_teams = {}
gb = player_info_df.groupby("Tm")
for x in gb.groups:
    players_teams[x] = gb.get_group(x)

In [12]:
#Convert the player into the index look-up list
player_idx = player_info_df['Player'].values.tolist() 
roster_idx = {}
for team in players_teams:
    roster_idx[team] = [player_idx.index(i) for i in players_teams[team]['Player'].values]

In [13]:
guest_teams = game16_df['Visitor/Neutral'].values 
host_teams = game16_df['Home/Neutral'].values 

In [14]:
player_num = len(player_idx)

In [15]:
#Convert guest team to an array of player's indicator
guest_team_line_up = []
for team in guest_teams:
    # Initialize the binary number
    player_indicator = np.array([False] * player_num)
    # Here I have the index of player
    list_idx = roster_idx[team]
    for i in list_idx:
        player_indicator[i] = True
    guest_team_line_up.append(player_indicator)
print "we have total:",len(guest_team_line_up)
print "confirm the previous num", len(guest_teams)

we have total: 1230
confirm the previous num 1230


In [16]:
#Convert guest team to an array of player's indicator
host_team_line_up = []
for team in host_teams:
    # Initialize the binary number
    player_indicator = np.array([False] * player_num)
    # Here I have the index of player
    list_idx = roster_idx[team]
    for i in list_idx:
        player_indicator[i] = True 
    host_team_line_up.append(player_indicator)
#     print len(beta)
#     print len(host_team_line_up[0])
#     check = beta[host_team_line_up[0]]
#     break
print "we have total:",len(host_team_line_up)
print "confirm the previous num", len(host_teams)

we have total: 1230
confirm the previous num 1230


In [17]:
with open("clean-data/guest_team_line_up.pkl", "wb") as f:
    pickle.dump(guest_team_line_up, f)
    
with open("clean-data/host_team_line_up.pkl", "wb") as f:
    pickle.dump(host_team_line_up, f)

Example of how to use these:

In [18]:
player_info_df[guest_team_line_up[0]]

,Player,Tm,2015-16,DRtg,ORtg
23,Tobias Harris,Detroit Pistons,16000000,107,120
39,Reggie Jackson,Detroit Pistons,14000000,108,109
117,Aron Baynes,Detroit Pistons,6500000,104,114
122,Jodie Meeks,Detroit Pistons,6270000,111,111
141,Josh Smith,Detroit Pistons,5400000,109,90
156,Marcus Morris,Detroit Pistons,5000000,108,107
209,Andre Drummond,Detroit Pistons,3272091,98,103
225,Anthony Tolliver,Detroit Pistons,3000000,107,113
229,Kentavious Caldwell-Pope,Detroit Pistons,2891760,107,107
235,Stanley Johnson,Detroit Pistons,2841960,106,91


In [19]:
player_info_df[host_team_line_up[0]]

,Player,Tm,2015-16,DRtg,ORtg
15,Paul Millsap,Atlanta Hawks,19000000,96,108
61,Al Horford,Atlanta Hawks,12000000,101,113
87,Tiago Splitter,Atlanta Hawks,8500000,102,113
96,Jeff Teague,Atlanta Hawks,8000000,104,105
127,Kyle Korver,Atlanta Hawks,5746479,105,104
184,Thabo Sefolosha,Atlanta Hawks,4000000,100,108
207,Mike Scott,Atlanta Hawks,3333333,104,112
230,Kirk Hinrich,Atlanta Hawks,2870000,110,103
280,Kent Bazemore,Atlanta Hawks,2000000,100,99
287,Dennis Schroder,Atlanta Hawks,1763400,103,97
